# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pickle
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# models
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import pandas as pd
import numpy as np
import nltk

In [5]:
# Uncomment these to download the packages

# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [4]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('messages', engine)
stop_words = stopwords.words()

X = df.message
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    #remove punctuation
    text_clean = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # normalize and tokenize
    tokens = word_tokenize(text_clean.lower())
    
    # init lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # remove stop words and lematize
    clean_tokens = []
    for token in tokens:
        clean_tokens = [lemmatizer.lemmatize(word.strip()) for word in tokens if word not in stop_words]

    return clean_tokens

tokens = tokenize(df.message[0])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ]))

    ])),

    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
# fit the pipeline
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# make prediction
print('Predicting ...')
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

Predicting ...
                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      6532
               request       0.73      0.51      0.60      1510
                 offer       0.00      0.00      0.00        48
           aid_related       0.75      0.61      0.67      3578
          medical_help       0.60      0.26      0.37       682
      medical_products       0.65      0.30      0.41       429
     search_and_rescue       0.54      0.16      0.25       233
              security       0.28      0.05      0.08       142
              military       0.60      0.29      0.39       283
                 water       0.73      0.65      0.69       514
                  food       0.81      0.67      0.73       943
               shelter       0.75      0.50      0.60       752
              clothing       0.66      0.30      0.41       118
                 money       0.52      0.29      0.37       206
        missing_people  

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline_fitted.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__count_vectorizer', 'features__text_pipeline__tfidf_transformer', 'features__text_pipeline__count_vectorizer__analyzer', 'features__text_pipeline__count_vectorizer__binary', 'features__text_pipeline__count_vectorizer__decode_error', 'features__text_pipeline__count_vectorizer__dtype', 'features__text_pipeline__count_vectorizer__encoding', 'features__text_pipeline__count_vectorizer__input', 'features__text_pipeline__count_vectorizer__lowercase', 'features__text_pipeline__count_vectorizer__max_df', 'features__text_pipeline__count_vectorizer__max_features', 'features__text_pipeline__count_vectorizer__min_df', 'features__text_pipeline__count_vectorizer__ngram_range', '

In [11]:
parameters = {'classifier__estimator__learning_rate': [0.01, 0.05],
              'classifier__estimator__n_estimators': [20, 50]}

cv = GridSearchCV(
    pipeline,
    param_grid=parameters,
    scoring='f1_micro',
    n_jobs=-1
)

print('Fitting...')
cv.fit(X_train, y_train)

# Get the predictions
print('Predicting ...')
y_prediction_test = cv.predict(X_test)

Fitting...
Predicting ...


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      6532
               request       0.84      0.18      0.29      1510
                 offer       0.00      0.00      0.00        48
           aid_related       0.78      0.27      0.40      3578
          medical_help       0.65      0.02      0.04       682
      medical_products       0.83      0.01      0.02       429
     search_and_rescue       0.75      0.01      0.03       233
              security       0.00      0.00      0.00       142
              military       0.50      0.00      0.01       283
                 water       0.66      0.70      0.68       514
                  food       0.76      0.69      0.72       943
               shelter       0.83      0.30      0.44       752
              clothing       0.83      0.08      0.15       118
                 money       0.00      0.00      0.00       206
        missing_people       0.33      

/Users/my_love/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))